In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt 

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Parameter

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 100000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 256
BATCH_SIZE = 64
LAYERS = 1

In [4]:
class  CharacterTable (object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

## Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    big_number = max([a,b])
    small_number = min([a,b])
    q = '{}-{}'.format(big_number, small_number)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(big_number - small_number)
    ans += ' ' * (DIGITS - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 100000


In [8]:
print(questions[:5], expected[:5])

['59-5   ', '62-34  ', '34-0   ', '54-5   ', '78-16  '] ['54 ', '28 ', '34 ', '49 ', '62 ']


## Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS )

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:70000]
train_y = y[:70000]
test_x = x[70000:]
test_y = y[70000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(63000, 7, 12)
(63000, 3, 12)
Validation Data:
(7000, 7, 12)
(7000, 3, 12)
Testing Data:
(30000, 7, 12)
(30000, 3, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False  True False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False  True
   False]
  [False False False False False False False False False  True False
   False]
  [False False  True False False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False  True False False False Fal

## Build Model

In [12]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               275456    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 256)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 256)            525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 3, 12)             3084      
_________________________________________________________________
activation_1 (Activation)    (None, 3, 12)             0         
Total params: 803,852
Trainable params: 803,852
Non-trainable params: 0
_________________________________________________________________


## Training

In [13]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    early_stopping_monitor = EarlyStopping(patience=5, mode='auto')
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val),
              callbacks=[early_stopping_monitor])
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 55s 878us/step - loss: 1.9664 - acc: 0.2668 - val_loss: 1.6519 - val_acc: 0.3842
Q 853-240 T 613 ☒ 688
Q 753-304 T 449 ☒ 338
Q 279-68  T 211 ☒ 188
Q 745-38  T 707 ☒ 708
Q 895-45  T 850 ☒ 835
Q 729-106 T 623 ☒ 666
Q 61-36   T 25  ☒ 31 
Q 270-157 T 113 ☒ 11 
Q 151-3   T 148 ☒ 131
Q 467-57  T 410 ☒ 436

--------------------------------------------------
Iteration 1
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 52s 830us/step - loss: 1.4989 - acc: 0.4372 - val_loss: 1.3508 - val_acc: 0.4924
Q 328-257 T 71  ☒ 10 
Q 150-46  T 104 ☒ 10 
Q 25-7    T 18  ☒ 1  
Q 818-299 T 519 ☒ 548
Q 817-539 T 278 ☒ 284
Q 452-42  T 410 ☒ 414
Q 335-72  T 263 ☒ 244
Q 924-53  T 871 ☒ 875
Q 112-17  T 95  ☒ 90 
Q 858-8   T 850 ☒ 858

--------------------------------------------------
Iterat

63000/63000 [==============================] - 58s 914us/step - loss: 0.0659 - acc: 0.9816 - val_loss: 0.0549 - val_acc: 0.9859
Q 190-92  T 98  ☑ 98 
Q 279-70  T 209 ☑ 209
Q 763-231 T 532 ☑ 532
Q 377-29  T 348 ☑ 348
Q 417-29  T 388 ☑ 388
Q 160-3   T 157 ☑ 157
Q 712-6   T 706 ☑ 706
Q 109-97  T 12  ☑ 12 
Q 738-15  T 723 ☑ 723
Q 986-634 T 352 ☑ 352

--------------------------------------------------
Iteration 16
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 57s 900us/step - loss: 0.0607 - acc: 0.9831 - val_loss: 0.0904 - val_acc: 0.9698
Q 527-83  T 444 ☑ 444
Q 787-597 T 190 ☒ 180
Q 228-135 T 93  ☑ 93 
Q 509-71  T 438 ☑ 438
Q 925-34  T 891 ☑ 891
Q 13-2    T 11  ☑ 11 
Q 911-29  T 882 ☑ 882
Q 979-446 T 533 ☑ 533
Q 996-490 T 506 ☑ 506
Q 739-97  T 642 ☑ 642

--------------------------------------------------
Iteration 17
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 58s 918us

63000/63000 [==============================] - 56s 888us/step - loss: 0.0292 - acc: 0.9921 - val_loss: 0.0467 - val_acc: 0.9848
Q 375-68  T 307 ☑ 307
Q 532-23  T 509 ☑ 509
Q 430-57  T 373 ☑ 373
Q 645-151 T 494 ☑ 494
Q 512-464 T 48  ☑ 48 
Q 781-697 T 84  ☑ 84 
Q 609-62  T 547 ☑ 547
Q 803-53  T 750 ☑ 750
Q 459-305 T 154 ☑ 154
Q 45-35   T 10  ☑ 10 

--------------------------------------------------
Iteration 31
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 894us/step - loss: 0.0277 - acc: 0.9923 - val_loss: 0.0161 - val_acc: 0.9959
Q 809-692 T 117 ☑ 117
Q 299-42  T 257 ☑ 257
Q 401-324 T 77  ☑ 77 
Q 964-360 T 604 ☑ 604
Q 918-155 T 763 ☑ 763
Q 861-49  T 812 ☑ 812
Q 765-536 T 229 ☑ 229
Q 505-389 T 116 ☑ 116
Q 877-214 T 663 ☑ 663
Q 245-21  T 224 ☑ 224

--------------------------------------------------
Iteration 32
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 886us

63000/63000 [==============================] - 56s 885us/step - loss: 0.0173 - acc: 0.9953 - val_loss: 0.0139 - val_acc: 0.9960
Q 874-621 T 253 ☑ 253
Q 848-69  T 779 ☑ 779
Q 702-34  T 668 ☑ 668
Q 917-212 T 705 ☑ 705
Q 721-67  T 654 ☑ 654
Q 937-55  T 882 ☑ 882
Q 579-5   T 574 ☑ 574
Q 714-18  T 696 ☑ 696
Q 729-443 T 286 ☑ 286
Q 229-89  T 140 ☑ 140

--------------------------------------------------
Iteration 46
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 888us/step - loss: 0.0086 - acc: 0.9982 - val_loss: 0.1108 - val_acc: 0.9661
Q 103-64  T 39  ☑ 39 
Q 690-531 T 159 ☑ 159
Q 476-88  T 388 ☑ 388
Q 756-16  T 740 ☑ 740
Q 552-452 T 100 ☑ 100
Q 691-76  T 615 ☑ 615
Q 902-7   T 895 ☑ 895
Q 412-3   T 409 ☑ 409
Q 263-28  T 235 ☑ 235
Q 792-21  T 771 ☑ 771

--------------------------------------------------
Iteration 47
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 890us

63000/63000 [==============================] - 56s 894us/step - loss: 0.0178 - acc: 0.9951 - val_loss: 0.0180 - val_acc: 0.9949
Q 709-60  T 649 ☑ 649
Q 633-79  T 554 ☑ 554
Q 86-85   T 1   ☑ 1  
Q 442-9   T 433 ☑ 433
Q 593-0   T 593 ☑ 593
Q 229-89  T 140 ☑ 140
Q 495-20  T 475 ☑ 475
Q 584-444 T 140 ☑ 140
Q 732-92  T 640 ☑ 640
Q 559-37  T 522 ☑ 522

--------------------------------------------------
Iteration 61
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 893us/step - loss: 0.0047 - acc: 0.9993 - val_loss: 0.0065 - val_acc: 0.9982
Q 433-68  T 365 ☑ 365
Q 336-210 T 126 ☑ 126
Q 525-312 T 213 ☑ 213
Q 996-684 T 312 ☑ 312
Q 896-99  T 797 ☑ 797
Q 164-129 T 35  ☑ 35 
Q 992-162 T 830 ☑ 830
Q 949-50  T 899 ☑ 899
Q 99-18   T 81  ☑ 81 
Q 730-9   T 721 ☑ 721

--------------------------------------------------
Iteration 62
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 893us

63000/63000 [==============================] - 56s 889us/step - loss: 0.0119 - acc: 0.9968 - val_loss: 0.0055 - val_acc: 0.9985
Q 953-409 T 544 ☑ 544
Q 544-8   T 536 ☑ 536
Q 67-36   T 31  ☑ 31 
Q 498-130 T 368 ☑ 368
Q 547-245 T 302 ☑ 302
Q 346-26  T 320 ☑ 320
Q 958-61  T 897 ☑ 897
Q 414-218 T 196 ☑ 196
Q 775-9   T 766 ☑ 766
Q 782-85  T 697 ☑ 697

--------------------------------------------------
Iteration 76
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 889us/step - loss: 0.0062 - acc: 0.9985 - val_loss: 0.0587 - val_acc: 0.9823
Q 673-41  T 632 ☑ 632
Q 539-22  T 517 ☑ 517
Q 412-80  T 332 ☑ 332
Q 739-97  T 642 ☑ 642
Q 596-98  T 498 ☑ 498
Q 951-6   T 945 ☑ 945
Q 363-3   T 360 ☑ 360
Q 395-51  T 344 ☑ 344
Q 933-110 T 823 ☑ 823
Q 976-76  T 900 ☑ 900

--------------------------------------------------
Iteration 77
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 895us

63000/63000 [==============================] - 56s 896us/step - loss: 0.0086 - acc: 0.9976 - val_loss: 0.0041 - val_acc: 0.9992
Q 642-7   T 635 ☑ 635
Q 410-131 T 279 ☑ 279
Q 423-3   T 420 ☑ 420
Q 274-99  T 175 ☑ 175
Q 400-107 T 293 ☑ 293
Q 164-129 T 35  ☑ 35 
Q 428-83  T 345 ☑ 345
Q 454-18  T 436 ☑ 436
Q 681-78  T 603 ☑ 603
Q 982-83  T 899 ☑ 899

--------------------------------------------------
Iteration 91
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 887us/step - loss: 8.1768e-04 - acc: 1.0000 - val_loss: 0.0037 - val_acc: 0.9991
Q 908-709 T 199 ☑ 199
Q 125-38  T 87  ☑ 87 
Q 772-126 T 646 ☑ 646
Q 858-493 T 365 ☑ 365
Q 390-6   T 384 ☑ 384
Q 893-640 T 253 ☑ 253
Q 973-78  T 895 ☑ 895
Q 123-48  T 75  ☑ 75 
Q 428-46  T 382 ☑ 382
Q 498-93  T 405 ☑ 405

--------------------------------------------------
Iteration 92
Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 56s 8

## Validation

In [14]:
right = 0
preds = model.predict_classes(test_x, verbose=0)

for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

Q 801-120 T 681 ☑ 681
Q 550-94  T 456 ☑ 456
Q 726-186 T 540 ☑ 540
Q 402-313 T 89  ☑ 89 
Q 988-153 T 835 ☑ 835
Q 154-132 T 22  ☑ 22 
Q 757-526 T 231 ☑ 231
Q 755-582 T 173 ☑ 173
Q 483-94  T 389 ☑ 389
Q 382-49  T 333 ☑ 333
Q 719-425 T 294 ☑ 294
Q 476-19  T 457 ☑ 457
Q 758-2   T 756 ☑ 756
Q 802-283 T 519 ☑ 519
Q 577-71  T 506 ☑ 506
Q 714-260 T 454 ☑ 454
Q 903-643 T 260 ☑ 260
Q 753-420 T 333 ☑ 333
Q 188-10  T 178 ☑ 178
Q 284-95  T 189 ☑ 189
Q 330-87  T 243 ☑ 243
Q 255-47  T 208 ☑ 208
Q 188-88  T 100 ☑ 100
Q 968-2   T 966 ☑ 966
Q 368-2   T 366 ☑ 366
Q 378-42  T 336 ☑ 336
Q 654-24  T 630 ☑ 630
Q 731-539 T 192 ☑ 192
Q 930-98  T 832 ☑ 832
Q 502-62  T 440 ☑ 440
Q 501-379 T 122 ☑ 122
Q 497-16  T 481 ☑ 481
Q 994-752 T 242 ☑ 242
Q 134-81  T 53  ☑ 53 
Q 506-33  T 473 ☑ 473
Q 972-658 T 314 ☑ 314
Q 689-95  T 594 ☑ 594
Q 77-28   T 49  ☑ 49 
Q 955-753 T 202 ☑ 202
Q 962-207 T 755 ☑ 755
Q 199-56  T 143 ☑ 143
Q 436-2   T 434 ☑ 434
Q 734-195 T 539 ☑ 539
Q 827-525 T 302 ☑ 302
Q 939-527 T 412 ☑ 412
Q 828-624 

Q 535-271 T 264 ☑ 264
Q 711-73  T 638 ☑ 638
Q 681-72  T 609 ☑ 609
Q 71-67   T 4   ☑ 4  
Q 979-70  T 909 ☑ 909
Q 210-54  T 156 ☑ 156
Q 579-228 T 351 ☑ 351
Q 130-26  T 104 ☑ 104
Q 561-39  T 522 ☑ 522
Q 594-423 T 171 ☑ 171
Q 779-8   T 771 ☑ 771
Q 687-66  T 621 ☑ 621
Q 771-31  T 740 ☑ 740
Q 413-86  T 327 ☑ 327
Q 171-29  T 142 ☑ 142
Q 450-210 T 240 ☑ 240
Q 882-45  T 837 ☑ 837
Q 214-67  T 147 ☑ 147
Q 937-590 T 347 ☑ 347
Q 643-315 T 328 ☑ 328
Q 863-9   T 854 ☑ 854
Q 279-178 T 101 ☑ 101
Q 155-85  T 70  ☑ 70 
Q 733-6   T 727 ☑ 727
Q 576-16  T 560 ☑ 560
Q 683-9   T 674 ☑ 674
Q 274-178 T 96  ☑ 96 
Q 38-10   T 28  ☑ 28 
Q 868-206 T 662 ☑ 662
Q 443-51  T 392 ☑ 392
Q 512-98  T 414 ☑ 414
Q 572-56  T 516 ☑ 516
Q 402-1   T 401 ☑ 401
Q 343-6   T 337 ☑ 337
Q 990-52  T 938 ☑ 938
Q 688-243 T 445 ☑ 445
Q 887-63  T 824 ☑ 824
Q 973-0   T 973 ☑ 973
Q 831-171 T 660 ☑ 660
Q 448-125 T 323 ☑ 323
Q 955-207 T 748 ☑ 748
Q 854-51  T 803 ☑ 803
Q 908-254 T 654 ☑ 654
Q 864-4   T 860 ☑ 860
Q 169-150 T 19  ☑ 19 
Q 394-77  

Q 265-71  T 194 ☑ 194
Q 880-555 T 325 ☑ 325
Q 243-124 T 119 ☑ 119
Q 246-6   T 240 ☑ 240
Q 347-8   T 339 ☑ 339
Q 255-74  T 181 ☑ 181
Q 243-60  T 183 ☑ 183
Q 941-98  T 843 ☑ 843
Q 838-821 T 17  ☑ 17 
Q 426-44  T 382 ☑ 382
Q 828-0   T 828 ☑ 828
Q 491-30  T 461 ☑ 461
Q 667-177 T 490 ☑ 490
Q 911-641 T 270 ☑ 270
Q 897-127 T 770 ☑ 770
Q 825-160 T 665 ☑ 665
Q 952-603 T 349 ☑ 349
Q 653-74  T 579 ☑ 579
Q 218-124 T 94  ☑ 94 
Q 284-2   T 282 ☑ 282
Q 399-24  T 375 ☑ 375
Q 102-11  T 91  ☑ 91 
Q 794-99  T 695 ☑ 695
Q 889-23  T 866 ☑ 866
Q 594-30  T 564 ☑ 564
Q 334-59  T 275 ☑ 275
Q 63-59   T 4   ☑ 4  
Q 172-6   T 166 ☑ 166
Q 269-93  T 176 ☑ 176
Q 127-57  T 70  ☑ 70 
Q 631-248 T 383 ☑ 383
Q 688-41  T 647 ☑ 647
Q 483-52  T 431 ☑ 431
Q 731-135 T 596 ☑ 596
Q 644-11  T 633 ☑ 633
Q 492-103 T 389 ☑ 389
Q 786-2   T 784 ☑ 784
Q 977-256 T 721 ☑ 721
Q 668-592 T 76  ☑ 76 
Q 706-313 T 393 ☑ 393
Q 974-330 T 644 ☑ 644
Q 143-54  T 89  ☑ 89 
Q 672-65  T 607 ☑ 607
Q 369-105 T 264 ☑ 264
Q 312-171 T 141 ☑ 141
Q 99-63   

Q 461-19  T 442 ☑ 442
Q 20-14   T 6   ☑ 6  
Q 237-13  T 224 ☑ 224
Q 786-66  T 720 ☑ 720
Q 953-34  T 919 ☑ 919
Q 611-4   T 607 ☑ 607
Q 774-87  T 687 ☑ 687
Q 949-203 T 746 ☑ 746
Q 401-331 T 70  ☑ 70 
Q 939-728 T 211 ☑ 211
Q 880-535 T 345 ☑ 345
Q 846-412 T 434 ☑ 434
Q 880-57  T 823 ☑ 823
Q 271-48  T 223 ☑ 223
Q 843-73  T 770 ☑ 770
Q 630-305 T 325 ☑ 325
Q 779-32  T 747 ☑ 747
Q 366-239 T 127 ☑ 127
Q 629-97  T 532 ☑ 532
Q 482-4   T 478 ☑ 478
Q 691-608 T 83  ☑ 83 
Q 982-29  T 953 ☑ 953
Q 990-702 T 288 ☑ 288
Q 538-43  T 495 ☑ 495
Q 904-20  T 884 ☑ 884
Q 525-134 T 391 ☑ 391
Q 924-66  T 858 ☑ 858
Q 316-30  T 286 ☑ 286
Q 264-82  T 182 ☑ 182
Q 554-1   T 553 ☑ 553
Q 742-87  T 655 ☑ 655
Q 353-93  T 260 ☑ 260
Q 377-70  T 307 ☑ 307
Q 99-12   T 87  ☑ 87 
Q 527-90  T 437 ☑ 437
Q 540-308 T 232 ☑ 232
Q 734-648 T 86  ☑ 86 
Q 570-206 T 364 ☑ 364
Q 866-83  T 783 ☑ 783
Q 89-20   T 69  ☑ 69 
Q 620-261 T 359 ☑ 359
Q 732-89  T 643 ☑ 643
Q 919-219 T 700 ☑ 700
Q 732-392 T 340 ☑ 340
Q 407-38  T 369 ☑ 369
Q 357-49  

Q 729-53  T 676 ☑ 676
Q 776-560 T 216 ☑ 216
Q 140-45  T 95  ☑ 95 
Q 259-17  T 242 ☑ 242
Q 227-58  T 169 ☑ 169
Q 97-68   T 29  ☑ 29 
Q 852-717 T 135 ☑ 135
Q 621-179 T 442 ☑ 442
Q 39-3    T 36  ☑ 36 
Q 867-743 T 124 ☑ 124
Q 582-93  T 489 ☑ 489
Q 946-789 T 157 ☑ 157
Q 925-27  T 898 ☑ 898
Q 706-32  T 674 ☑ 674
Q 674-354 T 320 ☑ 320
Q 979-661 T 318 ☑ 318
Q 845-436 T 409 ☑ 409
Q 84-83   T 1   ☑ 1  
Q 725-435 T 290 ☑ 290
Q 528-20  T 508 ☑ 508
Q 655-91  T 564 ☑ 564
Q 675-427 T 248 ☑ 248
Q 709-43  T 666 ☑ 666
Q 914-16  T 898 ☑ 898
Q 427-9   T 418 ☑ 418
Q 455-133 T 322 ☑ 322
Q 247-65  T 182 ☑ 182
Q 205-80  T 125 ☑ 125
Q 859-313 T 546 ☑ 546
Q 180-127 T 53  ☑ 53 
Q 903-150 T 753 ☑ 753
Q 868-80  T 788 ☑ 788
Q 652-298 T 354 ☑ 354
Q 374-64  T 310 ☑ 310
Q 889-18  T 871 ☑ 871
Q 917-42  T 875 ☑ 875
Q 532-36  T 496 ☑ 496
Q 412-279 T 133 ☑ 133
Q 916-190 T 726 ☑ 726
Q 554-402 T 152 ☑ 152
Q 493-24  T 469 ☑ 469
Q 324-8   T 316 ☑ 316
Q 673-80  T 593 ☑ 593
Q 66-13   T 53  ☑ 53 
Q 301-45  T 256 ☑ 256
Q 913-556 

...

Q 983-186 T 797 ☑ 797
Q 658-92  T 566 ☑ 566
Q 422-42  T 380 ☑ 380
Q 667-531 T 136 ☑ 136
Q 617-102 T 515 ☑ 515
Q 975-593 T 382 ☑ 382
Q 65-20   T 45  ☑ 45 
Q 889-830 T 59  ☑ 59 
Q 788-176 T 612 ☑ 612
Q 669-377 T 292 ☑ 292
Q 978-3   T 975 ☑ 975
Q 226-77  T 149 ☑ 149
Q 556-0   T 556 ☑ 556
Q 328-86  T 242 ☑ 242
Q 212-95  T 117 ☑ 117
Q 927-34  T 893 ☑ 893
Q 724-24  T 700 ☑ 700
Q 633-40  T 593 ☑ 593
Q 896-169 T 727 ☑ 727
Q 492-333 T 159 ☑ 159
Q 937-608 T 329 ☑ 329
Q 727-91  T 636 ☑ 636
Q 733-272 T 461 ☑ 461
Q 652-12  T 640 ☑ 640
Q 883-479 T 404 ☑ 404
Q 463-16  T 447 ☑ 447
Q 679-442 T 237 ☑ 237
Q 971-578 T 393 ☑ 393
Q 661-498 T 163 ☑ 163
Q 866-1   T 865 ☑ 865
Q 922-58  T 864 ☑ 864
Q 839-265 T 574 ☑ 574
Q 795-10  T 785 ☑ 785
Q 609-567 T 42  ☑ 42 
Q 538-8   T 530 ☑ 530
Q 763-9   T 754 ☑ 754
Q 197-10  T 187 ☑ 187
Q 906-803 T 103 ☑ 103
Q 745-84  T 661 ☑ 661
Q 523-79  T 444 ☑ 444
Q 967-295 T 672 ☑ 672
Q 723-2   T 721 ☑ 721
Q 953-716 T 237 ☑ 237
Q 491-74  T 417 ☑ 417
Q 730-685 T 45  ☑ 45 
Q 194-88  

Q 526-3   T 523 ☑ 523
Q 399-77  T 322 ☑ 322
Q 270-27  T 243 ☑ 243
Q 295-5   T 290 ☑ 290
Q 540-421 T 119 ☑ 119
Q 178-18  T 160 ☑ 160
Q 245-9   T 236 ☑ 236
Q 509-96  T 413 ☑ 413
Q 719-202 T 517 ☑ 517
Q 344-224 T 120 ☑ 120
Q 699-634 T 65  ☑ 65 
Q 755-43  T 712 ☑ 712
Q 834-58  T 776 ☑ 776
Q 491-97  T 394 ☑ 394
Q 306-300 T 6   ☒ 5  
Q 984-14  T 970 ☑ 970
Q 675-105 T 570 ☑ 570
Q 277-87  T 190 ☑ 190
Q 167-70  T 97  ☑ 97 
Q 564-78  T 486 ☑ 486
Q 884-77  T 807 ☑ 807
Q 118-25  T 93  ☑ 93 
Q 961-670 T 291 ☑ 291
Q 206-33  T 173 ☑ 173
Q 768-594 T 174 ☑ 174
Q 448-43  T 405 ☑ 405
Q 668-277 T 391 ☑ 391
Q 155-15  T 140 ☑ 140
Q 132-47  T 85  ☑ 85 
Q 158-11  T 147 ☑ 147
Q 595-37  T 558 ☑ 558
Q 424-222 T 202 ☑ 202
Q 403-34  T 369 ☑ 369
Q 806-731 T 75  ☑ 75 
Q 782-99  T 683 ☑ 683
Q 584-539 T 45  ☑ 45 
Q 795-275 T 520 ☑ 520
Q 841-53  T 788 ☑ 788
Q 445-311 T 134 ☑ 134
Q 568-400 T 168 ☑ 168
Q 240-81  T 159 ☑ 159
Q 766-115 T 651 ☑ 651
Q 853-176 T 677 ☑ 677
Q 892-76  T 816 ☑ 816
Q 958-278 T 680 ☑ 680
Q 901-780 

Q 927-45  T 882 ☑ 882
Q 593-47  T 546 ☑ 546
Q 81-53   T 28  ☑ 28 
Q 517-73  T 444 ☑ 444
Q 841-93  T 748 ☑ 748
Q 702-424 T 278 ☑ 278
Q 961-59  T 902 ☑ 902
Q 863-26  T 837 ☑ 837
Q 798-2   T 796 ☑ 796
Q 734-399 T 335 ☑ 335
Q 532-75  T 457 ☑ 457
Q 208-2   T 206 ☑ 206
Q 454-11  T 443 ☑ 443
Q 986-967 T 19  ☑ 19 
Q 34-12   T 22  ☑ 22 
Q 840-284 T 556 ☑ 556
Q 481-47  T 434 ☑ 434
Q 231-21  T 210 ☑ 210
Q 894-65  T 829 ☑ 829
Q 235-58  T 177 ☑ 177
Q 800-32  T 768 ☑ 768
Q 598-80  T 518 ☑ 518
Q 514-396 T 118 ☑ 118
Q 994-701 T 293 ☑ 293
Q 386-3   T 383 ☑ 383
Q 667-513 T 154 ☑ 154
Q 763-42  T 721 ☑ 721
Q 214-71  T 143 ☑ 143
Q 828-236 T 592 ☑ 592
Q 84-7    T 77  ☑ 77 
Q 513-94  T 419 ☑ 419
Q 305-231 T 74  ☑ 74 
Q 940-16  T 924 ☑ 924
Q 889-97  T 792 ☑ 792
Q 834-79  T 755 ☑ 755
Q 890-70  T 820 ☑ 820
Q 83-67   T 16  ☑ 16 
Q 936-268 T 668 ☑ 668
Q 924-76  T 848 ☑ 848
Q 179-27  T 152 ☑ 152
Q 227-49  T 178 ☑ 178
Q 524-21  T 503 ☑ 503
Q 205-79  T 126 ☑ 126
Q 985-47  T 938 ☑ 938
Q 820-811 T 9   ☑ 9  
Q 432-62  

Q 415-63  T 352 ☑ 352
Q 98-61   T 37  ☑ 37 
Q 675-664 T 11  ☑ 11 
Q 937-596 T 341 ☑ 341
Q 726-97  T 629 ☑ 629
Q 191-139 T 52  ☑ 52 
Q 582-441 T 141 ☑ 141
Q 174-89  T 85  ☑ 85 
Q 697-342 T 355 ☑ 355
Q 884-826 T 58  ☑ 58 
Q 910-71  T 839 ☑ 839
Q 432-29  T 403 ☑ 403
Q 856-309 T 547 ☑ 547
Q 575-299 T 276 ☑ 276
Q 594-91  T 503 ☑ 503
Q 733-321 T 412 ☑ 412
Q 261-23  T 238 ☑ 238
Q 975-685 T 290 ☑ 290
Q 441-46  T 395 ☑ 395
Q 881-34  T 847 ☑ 847
Q 779-93  T 686 ☑ 686
Q 562-22  T 540 ☑ 540
Q 404-103 T 301 ☑ 301
Q 475-229 T 246 ☑ 246
Q 201-59  T 142 ☑ 142
Q 313-93  T 220 ☑ 220
Q 708-0   T 708 ☑ 708
Q 103-35  T 68  ☑ 68 
Q 975-91  T 884 ☑ 884
Q 849-707 T 142 ☑ 142
Q 127-16  T 111 ☑ 111
Q 788-83  T 705 ☑ 705
Q 226-61  T 165 ☑ 165
Q 696-552 T 144 ☑ 144
Q 333-80  T 253 ☑ 253
Q 376-25  T 351 ☑ 351
Q 663-45  T 618 ☑ 618
Q 626-108 T 518 ☑ 518
Q 963-35  T 928 ☑ 928
Q 550-307 T 243 ☑ 243
Q 583-82  T 501 ☑ 501
Q 85-74   T 11  ☑ 11 
Q 990-45  T 945 ☑ 945
Q 198-65  T 133 ☑ 133
Q 156-49  T 107 ☑ 107
Q 266-25  

Q 502-64  T 438 ☑ 438
Q 78-40   T 38  ☑ 38 
Q 542-326 T 216 ☑ 216
Q 959-491 T 468 ☑ 468
Q 669-499 T 170 ☑ 170
Q 863-37  T 826 ☑ 826
Q 462-137 T 325 ☑ 325
Q 948-230 T 718 ☑ 718
Q 250-30  T 220 ☑ 220
Q 775-14  T 761 ☑ 761
Q 976-90  T 886 ☑ 886
Q 291-60  T 231 ☑ 231
Q 913-49  T 864 ☑ 864
Q 498-257 T 241 ☑ 241
Q 642-397 T 245 ☑ 245
Q 359-124 T 235 ☑ 235
Q 976-315 T 661 ☑ 661
Q 258-80  T 178 ☑ 178
Q 636-13  T 623 ☑ 623
Q 987-19  T 968 ☑ 968
Q 603-253 T 350 ☑ 350
Q 651-80  T 571 ☑ 571
Q 176-40  T 136 ☑ 136
Q 622-335 T 287 ☑ 287
Q 259-7   T 252 ☑ 252
Q 767-380 T 387 ☑ 387
Q 155-6   T 149 ☑ 149
Q 379-64  T 315 ☑ 315
Q 907-7   T 900 ☑ 900
Q 570-26  T 544 ☑ 544
Q 461-341 T 120 ☑ 120
Q 694-64  T 630 ☑ 630
Q 508-454 T 54  ☑ 54 
Q 466-14  T 452 ☑ 452
Q 836-29  T 807 ☑ 807
Q 814-657 T 157 ☑ 157
Q 644-10  T 634 ☑ 634
Q 542-57  T 485 ☑ 485
Q 773-403 T 370 ☑ 370
Q 400-169 T 231 ☑ 231
Q 926-225 T 701 ☑ 701
Q 272-34  T 238 ☑ 238
Q 669-541 T 128 ☑ 128
Q 489-345 T 144 ☑ 144
Q 224-27  T 197 ☑ 197
Q 725-25  